Import Library

In [1]:
import numpy as np
import cv2
import pandas as pd
import seaborn as sns
import os
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.optimizers import Adam, SGD, RMSprop
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
import scipy

Prepare Data Generator

In [17]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir= 'data/test/'

num_train = 28709
num_val = 3589
num_test = 3589
batch_size = 128
num_epoch = 20

train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_dir,
                                    target_size=(48, 48),
                                    batch_size=batch_size,
                                    shuffle=True,
                                    color_mode="grayscale",
                                    class_mode='categorical')

validation_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        val_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=False,
        color_mode="grayscale",
        class_mode='categorical')

test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(
        test_dir,
        target_size=(48, 48),
        batch_size=batch_size,
        shuffle=False,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [3]:
Model_Checkpoint_Callback = ModelCheckpoint('best_model.h5', 
                                            monitor='val_accuracy', 
                                            verbose=1, 
                                            save_best_only=True, 
                                            mode='max',
                                            save_freq = "epoch")
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', 
                              factor=0.75,
                              patience=5,
                              verbose=1,
                              mode='max')

Model Building

In [4]:
model = Sequential()

model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding = 'same',input_shape = (48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(256,(3,3),padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(4096))
model.add(Activation('relu'))

model.add(Dense(4096))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(7, activation='softmax'))
sgd = SGD(lr=0.01, weight_decay=0.0001, momentum=0.9, nesterov=True)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit_generator(generator=train_generator,
                              steps_per_epoch=num_train//batch_size,
                              epochs=num_epoch,
                              validation_data=validation_generator,
                              validation_steps=num_val//batch_size,
                              callbacks=[Model_Checkpoint_Callback,reduce_lr]
                              )

Epoch 1/20


C:\Users\Tommy\AppData\Local\Temp\ipykernel_15776\3403903270.py:47: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(generator=train_generator,


224/224 [==============================] - ETA: 0s - loss: 1.9589 - accuracy: 0.2649
Epoch 1: val_accuracy improved from -inf to 0.17439, saving model to best_model.h5


C:\Users\Tommy\AppData\Roaming\Python\Python38\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


224/224 [==============================] - 747s 3s/step - loss: 1.9589 - accuracy: 0.2649 - val_loss: 1.8744 - val_accuracy: 0.1744 - lr: 0.0100
Epoch 2/20
224/224 [==============================] - ETA: 0s - loss: 1.5921 - accuracy: 0.3657
Epoch 2: val_accuracy improved from 0.17439 to 0.29464, saving model to best_model.h5
224/224 [==============================] - 732s 3s/step - loss: 1.5921 - accuracy: 0.3657 - val_loss: 1.7604 - val_accuracy: 0.2946 - lr: 0.0100
Epoch 3/20
224/224 [==============================] - ETA: 0s - loss: 1.3499 - accuracy: 0.4784
Epoch 3: val_accuracy improved from 0.29464 to 0.47656, saving model to best_model.h5
224/224 [==============================] - 778s 3s/step - loss: 1.3499 - accuracy: 0.4784 - val_loss: 1.3323 - val_accuracy: 0.4766 - lr: 0.0100
Epoch 4/20
224/224 [==============================] - ETA: 0s - loss: 1.1878 - accuracy: 0.5454
Epoch 4: val_accuracy improved from 0.47656 to 0.50167, saving model to best_model.h5
224/224 [==========

Load Saved Model

In [25]:
from keras.models import load_model

model_test = load_model("best_model.h5")

Create Confusion Matrix

In [26]:
from sklearn.metrics import classification_report, confusion_matrix
Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)
y_pred = np.argmax(Y_pred, axis=1)
print('Confusion Matrix')
print(confusion_matrix(test_generator.classes, y_pred))

C:\Users\Tommy\AppData\Local\Temp\ipykernel_15776\4109819777.py:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  Y_pred = model_test.predict_generator(test_generator, num_test // batch_size+1)


Confusion Matrix
[[262   0  21  34  38 108   4]
 [ 12  22   2   4   3  13   0]
 [ 55   1 171  27  51 173  18]
 [ 31   0  11 721  60  66   6]
 [ 61   0  22  56 295 170   3]
 [ 83   0  29  38  76 423   4]
 [ 26   1  36  34  18  16 284]]


In [27]:
print('Classification Report')
target_names = list(train_generator.class_indices.keys())
print(classification_report(test_generator.classes, y_pred, target_names=target_names))

Classification Report
              precision    recall  f1-score   support

       angry       0.49      0.56      0.53       467
     disgust       0.92      0.39      0.55        56
        fear       0.59      0.34      0.43       496
       happy       0.79      0.81      0.80       895
     neutral       0.55      0.49      0.51       607
         sad       0.44      0.65      0.52       653
    surprise       0.89      0.68      0.77       415

    accuracy                           0.61      3589
   macro avg       0.67      0.56      0.59      3589
weighted avg       0.63      0.61      0.61      3589

